In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!pip install unrar
!unrar x /content/ExportedTemplateAMV6.rar /content/sample_data
import os
import pandas as pd


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/ExportedTemplateAMV6.rar

Creating    /content/sample_data/ExportedTemplateAM                   OK
Extracting  /content/sample_data/ExportedTemplateAM/Abdominal aortic aneurysm.xlsx       0%  OK 
Extracting  /content/sample_data/ExportedTemplateAM/Abdominal compartment syndrome.xlsx       0%  OK 
Extracting  /content/sample_data/ExportedTemplateAM/Abdominal Migraine.xlsx       0%  OK 
Extracting  /content/sample_data/ExportedTemplateAM/Abetalipoproteinaemia.xlsx       0%  OK 
Extracting  /content/sample_data/ExportedTemplateAM/Abnormal findings on cytological and histological examination of urine.xlsx       0%  OK 
Extracting  /content/sample_data/ExportedTemplateAM/Abnormal uterine bleeding.xlsx       0%  OK 
Extracting  /content/sample_data/ExportedTemplateAM/Abnormalities of size and form of teeth.xlsx       0%  OK 
Extra

In [2]:
path = "/content/sample_data/ExportedTemplateAM"
excel_files = [f for f in os.listdir(path) if f.endswith('.xlsx')]

dfs = [pd.read_excel(os.path.join(path, f)) for f in excel_files]
concatenated_df = pd.concat(dfs, axis=0, ignore_index=True)

In [ ]:
from tqdm import tqdm
import psutil

path = "/content/ExportedTemplatesCM"
excel_files = [f for f in os.listdir(path) if f.endswith('.xlsx')]

dfs = []
total_mem = 0
for f in tqdm(excel_files, desc="Reading files"):
    file_path = os.path.join(path, f)
    df = pd.read_excel(file_path)
    total_mem += psutil.Process(os.getpid()).memory_info().rss
    dfs.append(df)

concatenated_df = pd.concat(dfs, axis=0, ignore_index=True)

print(f"Total memory used: {total_mem / pow(1024, 2):.2f} MB")

Reading files: 100%|██████████| 71144/71144 [42:19<00:00, 28.02it/s]


Total memory used: 254373120.63 MB


In [3]:
concatenated_df.to_excel("concatenat_A.iGuide_templates_24_july_2023.xlsx", index=False)

In [ ]:
df = pd.read_excel('/content/concatenat_17_july_2023.xlsx', sheet_name='Sheet3')
df

,ICD_10_AM
0,J45.9
1,Z27.4
2,U83.3
3,N39.0
4,G43.8
...,...
4341,A83.6
4342,C10.8
4343,A83.8
4344,A00.0


In [ ]:
import pandas as pd

df

def get_parent_code(code):
    if '.' not in code:
        return f"{code}.00"
    else:
        split_code = code.split('.')
        if split_code[1] == '00' or split_code[1] == '0':
            return code
        else:
            return f"{split_code[0]}.00"

df['Parent_Code'] = df['ICD_10_AM'].apply(get_parent_code)

# Filter out rows where the child code is the same as the parent code
df = df[df['Parent_Code'] != df['ICD_10_AM']]

# Group by parent code and aggregate child codes into a list
grouped_df = df.groupby('Parent_Code')['ICD_10_AM'].apply(list).reset_index()

# Create a dataframe with child codes in separate columns
child_codes_df = pd.DataFrame(grouped_df['ICD_10_AM'].tolist())

# Concatenate the parent codes with child codes
parent_child_df = pd.concat([grouped_df['Parent_Code'], child_codes_df], axis=1)

# Update column names to avoid conflicts
parent_child_df.columns = ['Parent_Code'] + [f'Child_{i}' for i in range(1, len(parent_child_df.columns))]

# Fill None values with an empty string
parent_child_df.fillna('', inplace=True)

print(parent_child_df)

In [ ]:
parent_child_df

,Parent_Code,Child_1,Child_2,Child_3,Child_4,Child_5,Child_6,Child_7,Child_8,Child_9,...,Child_80,Child_81,Child_82,Child_83,Child_84,Child_85,Child_86,Child_87,Child_88,Child_89
0,A00.00,A00.9,A00.1,,,,,,,,...,,,,,,,,,,
1,A01.00,A01.4,A01.1,A01.2,A01.3,,,,,,...,,,,,,,,,,
2,A02.00,A02.1,A02.2,A02.8,A02.9,,,,,,...,,,,,,,,,,
3,A03.00,A03.1,A03.2,A03.3,A03.8,A03.9,,,,,...,,,,,,,,,,
4,A04.00,A04.9,A04.8,A04.1,A04.2,A04.3,A04.4,A04.5,A04.6,A04.7,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,Z71.00,Z71.6,,,,,,,,,...,,,,,,,,,,
859,Z85.00,Z85.5,Z85.8,,,,,,,,...,,,,,,,,,,
860,Z86.00,Z86.61,,,,,,,,,...,,,,,,,,,,
861,Z87.00,Z87.11,,,,,,,,,...,,,,,,,,,,


import numpy as np
from google.colab import autoviz
df_1223816423332546601 = autoviz.get_registered_df('df_1223816423332546601')

def categorical_histogram(df, colname, figsize=(2, 1.2), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  _, ax = plt.subplots(figsize=figsize)
  bars = df[colname].value_counts()
  plt.barh(bars.index, bars.values, color=sns.palettes.mpl_palette(mpl_palette_name))
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(df_1223816423332546601, *['Child_61'], **{})
chart

import numpy as np
from google.colab import autoviz
df_1223816423332546601 = autoviz.get_registered_df('df_1223816423332546601')

def categorical_histogram(df, colname, figsize=(2, 1.2), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  _, ax = plt.subplots(figsize=figsize)
  bars = df[colname].value_counts()
  plt.barh(bars.index, bars.values, color=sns.palettes.mpl_palette(mpl_palette_name))
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(df_1223816423332546601, *['Child_62'], **{})
chart

import numpy as np
from google.colab import autoviz
df_1223816423332546601 = autoviz.get_registered_df('df_1223816423332546601')

def categorical_histogram(df, colname, figsize=(2, 1.2), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  _, ax = plt.subplots(figsize=figsize)
  bars = df[colname].value_counts()
  plt.barh(bars.index, bars.values, color=sns.palettes.mpl_palette(mpl_palette_name))
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(df_1223816423332546601, *['Child_63'], **{})
chart

import numpy as np
from google.colab import autoviz
df_1223816423332546601 = autoviz.get_registered_df('df_1223816423332546601')

def categorical_histogram(df, colname, figsize=(2, 1.2), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  _, ax = plt.subplots(figsize=figsize)
  bars = df[colname].value_counts()
  plt.barh(bars.index, bars.values, color=sns.palettes.mpl_palette(mpl_palette_name))
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(df_1223816423332546601, *['Child_64'], **{})
chart

import numpy as np
from google.colab import autoviz
df_1223816423332546601 = autoviz.get_registered_df('df_1223816423332546601')

def heatmap(df, x_colname, y_colname, figsize=(2, 2), mpl_palette_name='viridis'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  import pandas as pd
  plt.subplots(figsize=figsize)
  df_2dhist = pd.DataFrame({
      x_label: grp[y_colname].value_counts()
      for x_label, grp in df.groupby(x_colname)
  })
  sns.heatmap(df_2dhist, cmap=mpl_palette_name)
  plt.xlabel(x_colname)
  plt.ylabel(y_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = heatmap(df_1223816423332546601, *['Child_61', 'Child_62'], **{})
chart

import numpy as np
from google.colab import autoviz
df_1223816423332546601 = autoviz.get_registered_df('df_1223816423332546601')

def heatmap(df, x_colname, y_colname, figsize=(2, 2), mpl_palette_name='viridis'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  import pandas as pd
  plt.subplots(figsize=figsize)
  df_2dhist = pd.DataFrame({
      x_label: grp[y_colname].value_counts()
      for x_label, grp in df.groupby(x_colname)
  })
  sns.heatmap(df_2dhist, cmap=mpl_palette_name)
  plt.xlabel(x_colname)
  plt.ylabel(y_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = heatmap(df_1223816423332546601, *['Child_62', 'Child_63'], **{})
chart

import numpy as np
from google.colab import autoviz
df_1223816423332546601 = autoviz.get_registered_df('df_1223816423332546601')

def heatmap(df, x_colname, y_colname, figsize=(2, 2), mpl_palette_name='viridis'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  import pandas as pd
  plt.subplots(figsize=figsize)
  df_2dhist = pd.DataFrame({
      x_label: grp[y_colname].value_counts()
      for x_label, grp in df.groupby(x_colname)
  })
  sns.heatmap(df_2dhist, cmap=mpl_palette_name)
  plt.xlabel(x_colname)
  plt.ylabel(y_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = heatmap(df_1223816423332546601, *['Child_63', 'Child_64'], **{})
chart

import numpy as np
from google.colab import autoviz
df_1223816423332546601 = autoviz.get_registered_df('df_1223816423332546601')

def heatmap(df, x_colname, y_colname, figsize=(2, 2), mpl_palette_name='viridis'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  import pandas as pd
  plt.subplots(figsize=figsize)
  df_2dhist = pd.DataFrame({
      x_label: grp[y_colname].value_counts()
      for x_label, grp in df.groupby(x_colname)
  })
  sns.heatmap(df_2dhist, cmap=mpl_palette_name)
  plt.xlabel(x_colname)
  plt.ylabel(y_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = heatmap(df_1223816423332546601, *['Child_64', 'Child_65'], **{})
chart

In [ ]:
parent_child_df.to_excel('parent.xlsx')

In [ ]:
# Install Biopython
!pip install biopython

# Import necessary modules
from Bio import Entrez

# Always tell NCBI who you are
Entrez.email = "andalusiamedicalrecords@gmail.com"

In [ ]:
# Define your search term
search_term = "Eclampsia"

# Use Entrez.esearch to get ids of articles related to the search term
handle = Entrez.esearch(db="pubmed", term=search_term, retmax=5)
record = Entrez.read(handle)
handle.close()

idlist = record["IdList"]

# Fetch details of each article in the idlist
for article_id in idlist:
    handle = Entrez.efetch(db="pubmed", id=article_id, rettype="abstract", retmode="text")
    print(handle.read())
    handle.close()

In [ ]:
import pandas as pd
# Define your search term
search_term = "Eclampsia"

# Use Entrez.esearch to get ids of articles related to the search term
handle = Entrez.esearch(db="pubmed", term=search_term, retmax=5)
record = Entrez.read(handle)
handle.close()

idlist = record["IdList"]

# Initialize a list to store dictionaries containing article details
articles = []

# Fetch details of each article in the idlist
for article_id in idlist:
    handle = Entrez.efetch(db="pubmed", id=article_id, rettype="abstract", retmode="text")
    abstract = handle.read()
    handle.close()

    # Construct the URL for the abstract
    url = f"https://pubmed.ncbi.nlm.nih.gov/{article_id}/"

    # Create a dictionary with article details and append to the list
    articles.append({'id': article_id, 'abstract': abstract, 'url': url})

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(articles)

# Print the DataFrame
df